In [1]:
import os
import cv2
import numpy as np
import mediapipe as mp
import zarr

## Funciones Utiles

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [3]:
def mediapipe_detection(frame, model):
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    frame_rgb.flags.writeable = False                  # Imagen no editable
    results = model.process(frame_rgb)                 # Make prediction
    return results

In [4]:
def draw_landmarks(image, results):
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS
                             #mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             #mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
    #                         ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS 
                             #mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             #mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS 
                             #mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             #mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2
                             )

In [ ]:
# width = cap.get(3)
# height = cap.get(4)
# fps = cap.get(5)
# print('alto:', height, 'ancho:', width, 'fps:', fps)

In [51]:
# Prueba
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
# Set mediapipe model 
with mp_holistic.Holistic(
    #static_image_mode=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read camera
        ret, frame = cap.read()
        w, h = frame.shape[1], frame.shape[0]
        print(w, h)
        # Make detections
        results = mediapipe_detection(frame, holistic)
        # Draw landmarks
        draw_landmarks(frame, results)
        # Show to screen
        cv2.imshow('Camara Frame', frame)

        # Break gracefully
        t = cv2.waitKey(1)
        if t == ord('q') or t == 27:
            break
    cap.release()
    cv2.destroyAllWindows()

640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480
640 480


In [8]:
# # prueba 2
# len(results.right_hand_landmarks.landmark)

### Extraccion de puntos y procesamiento de las coordenadas

In [5]:
from math import dist, atan2, pi

In [6]:
#%%timeit
#len([coord for idx in pose_list for coord in (results.pose_landmarks.landmark[idx].x, results.pose_landmarks.landmark[idx].y)])

In [7]:
#len([coord for landmark in results.right_hand_landmarks.landmark for coord in (landmark.x, landmark.y)] if results.right_hand_landmarks else [0]*42)

In [9]:
def extract_keypoints(results): # distancias y angulos
    pose_list = [11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
    x_minl = min([landmark.x for landmark in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else 0
    y_minl = min([landmark.y for landmark in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else 0
    x_minr = min([landmark.x for landmark in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else 0
    y_minr = min([landmark.y for landmark in results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else 0
    
    lh = [coord for landmark in results.left_hand_landmarks.landmark for coord in (landmark.x - x_minl, landmark.y - y_minl)] \
            if results.left_hand_landmarks else [0]*42 # distancias entre los puntos minimos de la mano izquierda y los demas dedos
    rh = [coord for landmark in results.right_hand_landmarks.landmark for coord in (landmark.x - x_minr, landmark.y - y_minr)] \
            if results.right_hand_landmarks else [0]*42 # distancias entre los puntos minimos de la mano derecha y los demas dedos
    c_nx = results.pose_landmarks.landmark[0].x if results.pose_landmarks else 0.5 
    c_ny = results.pose_landmarks.landmark[0].y if results.pose_landmarks else 0.2 # coordenada de la nariz, punto de referecia origen
    distancias_rh = [dist((landmark.x, landmark.y), (c_nx, c_ny)) for landmark in results.right_hand_landmarks.landmark] \
            if results.right_hand_landmarks else [0]*21 # distancia de cada dedo al punto de referencia
    distancias_lh = [dist((landmark.x, landmark.y), (c_nx, c_ny)) for landmark in results.left_hand_landmarks.landmark] \
            if results.left_hand_landmarks else [0]*21 
    angulos = [atan2((results.pose_landmarks.landmark[idx].y - c_ny), (results.pose_landmarks.landmark[idx].x - c_nx))/(2*pi) for idx in pose_list] \
            if results.pose_landmarks else [0]*12 # angulos del hombro, antebrazo, brazo y la muñeca con respecto al punto de referencia
    # 138 features en total
    return rh + distancias_rh + angulos + lh + distancias_lh

In [12]:
# test
result_test = extract_keypoints(results)

In [13]:
len(result_test)

138

## Configuraciónes

In [10]:
carpeta= "Datos_lsb"
# Verificar si la carpeta ya existe
if not os.path.exists(carpeta):
    os.mkdir(carpeta)
    print(f"La carpeta '{carpeta}' ha sido creada.")
else:
    print(f"La carpeta '{carpeta}' ya existe")

La carpeta 'Datos_lsb' ya existe


In [125]:
# Clases
signos = {
    #0: 'O' #,
    #1: '1' #, 
    #2: '2' #, 
    #3: '3' #,
    #4: '4'
    #5: '5'
    #6: '6' #, 
    #7: '7'#, 
    #8: '8' #,
    #9: '9'
    #10: '10' 
    #11: 'A' #, 
    #12: 'B' #, 
    #13: 'C' #, 
    #14: 'D' #, 
    #15: 'E' #, 
    #16: 'F' #, 
    #17: 'G' #,
    #18: 'H' #, 
    #19: 'I' 
    #20: 'J' #--cam 20
    #21: 'K' #, 
    #22: 'L' 
    #23: 'LL' #
    #24: 'M'#, 
    #25: 'N'
    #26: 'eNie' 
    #27: 'P'
    #28: 'Q' #, -- cam
    #29: 'R' #, 
    #30: 'S'
    #31: 'T'
    #32: 'U' #,
    #33: 'V' #, 
    #34: 'W' #,
    #35: 'X' #, 
    #36: 'Y'
    #37: 'Z' #
    #38: 'espacio' # espacio, 
    #39: 'HOLA' #-- 
    #40: 'BUENOS DIAS' #-
    #41: 'GRACIAS' #,
    #42 : 'POR FAVOR',
    #43: 'YO'
    #44: 'ABOGADO'
    #45: 'DOCTOR'
    #46: 'ELECTRICISTA'
    #47: 'POLICIA'
    #48: 'PRESIDENTE'
    #49: 'PROFESOR'
    #50: 'NOMBRE'
    #51: 'ES' #ser/estar
    #52: 'MI'  # ------
    #53: 'TRABAJO'
    #54: 'LA PAZ'
    #55: 'SUCRE'
    #56: 'COCHABAMBA'
    57: 'VIVO'
}

# Número de videos o secuencias
num_secuencias = 13
# frames por cada video: 30 frames por secuencia
longitud_secuencia = 30 # frames

## Recolección de datos

In [126]:
# datos
data = []
label = []
# camara 1
cap = cv2.VideoCapture(0, cv2.CAP_DSHOW) # 0 ivcam celular, 1 camara hp

# inicio de mediapipe model 
with mp_holistic.Holistic(
    #static_image_mode=True,
    min_detection_confidence=0.5, 
    min_tracking_confidence=0.5) as holistic:
    # Loop through actions
    for key, value in signos.items():
        # Loop through sequences aka videos
        for sequence in range(num_secuencias):
            # Loop through video length aka sequence length
            data_aux = []
            for frame_num in range(longitud_secuencia + 3):
                # lectura camara
                ret, frame = cap.read()
                # Deteccion de los puntos de las manos y pose
                results = mediapipe_detection(frame, holistic)
                # Dibuja los Landmarks
                draw_landmarks(frame, results)
                # logica de espera
                if frame_num == 2: 
                    cv2.putText(frame, f'Prep - Letra {value}', (15,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    cv2.putText(frame, f'Secuencia {sequence+1}', (15,60), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Frame', frame)
                    cv2.waitKey(5000)
                if frame_num > 2:
                    cv2.putText(frame, f'Ahora Letra {value}', (15,30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('Frame', frame)
                #if frame_num != 0:
                    # extraer puntos o coordenadas y angulos
                    keypoints = extract_keypoints(results) # 138 
                    #data_aux = list(data_aux + keypoints) # 114*30
                    data_aux.append(keypoints) # 30*138
                # Break
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            data.append(data_aux)
            label.append(key)
    cap.release()
    cv2.destroyAllWindows()

## Guardado de los datos

In [127]:
# verificación forma del array datoss secuenciales
np.array(data).shape

(13, 30, 138)

In [128]:
np.unique(label)

array([57])

In [12]:
# Conversion a numpy array por primera vez
# data_array = np.array(data)
# labels_array = np.array(label)
# print(f'Tamaño de los datos rec: {data_array.shape}, labels: {labels_array.shape}')

In [28]:
# Guardado de los datos en formato de numpy arrays por primera vez
# npy_path = os.path.join(carpeta,'lsb_array')
# npy_pathlb = os.path.join(carpeta,'lsb_labels_array')
# np.save(npy_path, data_array)
# np.save(npy_pathlb, labels_array)

In [107]:
# Carga de los datos zarr ya recolentados por primera vez

# data_array_1 = zarr.load('Datos_lsb/datos.zarr') # formato zarr numpy
# labels_array_1 = np.load('Datos_lsb/lsb_labels_array.npy')
# print(f'Tamaño de los datos: {data_array_1.shape}, labels: {labels_array_1.shape}')

Tamaño de los datos: (1870, 30, 138), labels: (1870,)


In [90]:
# #Carga de los datos ya recolentados por primera vez array 2
# data_array = zarr.load('Datos_lsb/datos2.zarr') # formato zarr numpy
# labels_array = np.load('Datos_lsb/lsb_labels_array_2.npy')
# print(f'Tamaño de los datos: {data_array.shape}, labels: {labels_array.shape}')

In [14]:
# #Carga de los datos ya recolentados por primera vez datos 3
# data_array = zarr.load('Datos_lsb/datos3.zarr') # formato zarr numpy
# labels_array = np.load('Datos_lsb/lsb_labels_array_3.npy')
# print(f'Tamaño de los datos: {data_array.shape}, labels: {labels_array.shape}')

Tamaño de los datos: (1785, 30, 138), labels: (1785,)


In [129]:
# AGREGACIÓN, para Nuevos datos o signos recolectados 
data_array = np.concatenate((data_array, np.array(data)), axis=0)
labels_array = np.concatenate((labels_array, np.array(label)), axis=0)
print(f'Nuevo Tamaño de los datos: {data_array.shape}, labels: {labels_array.shape}')

Nuevo Tamaño de los datos: (2060, 30, 138), labels: (2060,)


In [274]:
# data_array = data_array[:1235]
# labels_array = labels_array[:1235]

In [130]:
import pandas as pd

In [18]:
# Vista Cantidad de videos por cada letra o clase
# pd.Series(labels_array).value_counts().to_frame().rename_axis('Señas').rename(columns={'count': 'Secuencias'})

In [131]:
# Vista Cantidad de videos por cada letra o clase
pd.Series(labels_array).value_counts().to_frame().rename_axis('Señas').rename(columns={'count': 'Secuencias'})

,Secuencias
Señas,
57,50
48,50
39,50
41,50
42,50
43,50
45,50
46,50
47,50


In [132]:
pd.Series(labels_array).nunique()

56

In [27]:
# Guardado de los datos nuevamente, formato numpy, sobre escribir el archivo
# npy_path = os.path.join(carpeta,'lsb_array')
# npy_pathlb = os.path.join(carpeta,'lsb_labels_array')
# np.save(npy_path, data_array)
# np.save(npy_pathlb, labels_array)

In [30]:
# GUARDADO DE DATOS, formato zarr y npy 1
# path_labels = os.path.join(carpeta,'lsb_labels_array') # path
# zarr.save('Datos_lsb/datos.zarr', data_array_1) # datos o features
# np.save(path_labels, labels_array_1) # etiquetas o labels

In [18]:
# GUARDADO DE DATOS, formato zarr y npy 2
# path_labels = os.path.join(carpeta,'lsb_labels_array_2') # path
# zarr.save('Datos_lsb/datos2.zarr', data_array) # datos o features
# np.save(path_labels, labels_array) # etiquetas o labels

In [133]:
# GUARDADO DE DATOS, formato zarr y npy 3
path_labels = os.path.join(carpeta,'lsb_labels_array_3') # path
zarr.save('Datos_lsb/datos3.zarr', data_array) # datos o features
np.save(path_labels, labels_array) # etiquetas o labels

### Union de los datos procesados obtenidos

In [3]:
# Carga de los datos zarr  1 recolentados
# data_array_1 = zarr.load('Datos_lsb/datos.zarr')
# labels_array_1 = np.load('Datos_lsb/lsb_labels_array.npy')
# print(f'Tamaño de los datos 1: {data_array_1.shape}, labels: {labels_array_1.shape}')

Tamaño de los datos 1: (1870, 30, 138), labels: (1870,)


In [4]:
# Carga de los datos 2
data_array_2 = zarr.load('Datos_lsb/datos2.zarr')
labels_array_2 = np.load('Datos_lsb/lsb_labels_array_2.npy')
print(f'Tamaño de los datos 2: {data_array_2.shape}, labels: {labels_array_2.shape}')

Tamaño de los datos 2: (1110, 30, 138), labels: (1110,)


In [5]:
data_array = np.concatenate((data_array_1, data_array_2), axis=0)
labels_array = np.concatenate((labels_array_1, labels_array_2), axis=0)
print(f'Nuevo Tamaño de los datos total: {data_array.shape}, labels: {labels_array.shape}')

Nuevo Tamaño de los datos total: (2980, 30, 138), labels: (2980,)


In [8]:
# GUARDADO DE DATOS
# formato zarr
path_labels = os.path.join(carpeta,'lsb_etiquetas') # path
zarr.save('Datos_lsb/datos_lsb.zarr', data_array) # datos o features
np.save(path_labels, labels_array) # etiquetas o labels